# Long-Short Strategy, Part 5: Generating out-of-sample predictions

In this section, we'll start designing, implementing, and evaluating a trading strategy for US equities driven by daily return forecasts produced by gradient boosting models.

As in the previous examples, we'll lay out a framework and build a specific example that you can adapt to run your own experiments. There are numerous aspects that you can vary, from the asset class and investment universe to more granular aspects like the features, holding period, or trading rules. See, for example, the **Alpha Factor Library** in the [Appendix](../24_alpha_factor_library) for numerous additional features.

We'll keep the trading strategy simple and only use a single ML signal; a real-life application will likely use multiple signals from different sources, such as complementary ML models trained on different datasets or with different lookahead or lookback periods. It would also use sophisticated risk management, from simple stop-loss to value-at-risk analysis.

**Six notebooks** cover our workflow sequence:

1. [preparing_the_model_data](04_preparing_the_model_data.ipyny): we engineer a few simple features from the Quandl Wiki data 
2. [trading_signals_with_lightgbm_and_catboost](05_trading_signals_with_lightgbm_and_catboost.ipynb): we tune hyperparameters for LightGBM and CatBoost to select a model, using 2015/16 as our validation period. 
3. [evaluate_trading_signals](06_evaluate_trading_signals): we compare the cross-validation performance using various metrics to select the best model. 
4. [model_interpretation](07_model_interpretation.ipynb): we take a closer look at the drivers behind the best model's predictions.
5. `making_out_of_sample_predictions` (this noteboook): we predict returns for our out-of-sample period 2017.
6. [backtesting_with_zipline](09_backtesting_with_zipline.ipynb): evaluate the historical performance of a long-short strategy based on our predictive signals using Zipline.

## Imports & Settings

In [130]:
import warnings
warnings.filterwarnings('ignore')

In [131]:
%matplotlib inline

from time import time
import sys, os
from pathlib import Path

import pandas as pd
from scipy.stats import spearmanr

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [132]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import MultipleTimeSeriesCV

In [133]:
sns.set_style('whitegrid')

In [134]:
YEAR = 252
idx = pd.IndexSlice

In [135]:
scope_params = ['lookahead', 'train_length', 'test_length']
daily_ic_metrics = ['daily_ic_mean', 'daily_ic_mean_n', 'daily_ic_median', 'daily_ic_median_n']
lgb_train_params = ['learning_rate', 'num_leaves', 'feature_fraction', 'min_data_in_leaf']
catboost_train_params = ['max_depth', 'min_child_samples']

## Generate LightGBM predictions

### Model Configuration

In [136]:
base_params = dict(boosting='gbdt',
                   objective='regression',
                   verbose=-1)

categoricals = ['year', 'month', 'sector', 'weekday']

In [137]:
lookahead = 1
store = Path('data/predictions.h5')

### Get Data

In [138]:
data = pd.read_hdf('data.h5', 'model_data').sort_index()

In [139]:
labels = sorted(data.filter(like='_fwd').columns)
features = data.columns.difference(labels).tolist()
label = f'r{lookahead:02}_fwd'

In [140]:
data = data.loc[idx[:, '2010':], features + [label]].dropna()

In [141]:
for feature in categoricals:
    data[feature] = pd.factorize(data[feature], sort=True)[0]

In [142]:
lgb_data = lgb.Dataset(data=data[features],
                       label=data[label],
                       categorical_feature=categoricals,
                       free_raw_data=False)

### Generate predictions

In [143]:
lgb_ic = pd.read_hdf('data/model_tuning.h5', 'lgb/ic')
lgb_daily_ic = pd.read_hdf('data/model_tuning.h5', 'lgb/daily_ic')

In [144]:
def get_lgb_params(data, t=5, best=0):
    param_cols = scope_params[1:] + lgb_train_params + ['boost_rounds']
    df = data[data.lookahead==t].sort_values('ic', ascending=False).iloc[best]
    return df.loc[param_cols]

In [145]:
for position in range(10):
    params = get_lgb_params(lgb_daily_ic,
                            t=lookahead,
                            best=position)

    params = params.to_dict()

    for p in ['min_data_in_leaf', 'num_leaves']:
        params[p] = int(params[p])
    train_length = int(params.pop('train_length'))
    test_length = int(params.pop('test_length'))
    num_boost_round = int(params.pop('boost_rounds'))
    params.update(base_params)

    print(f'\nPosition: {position:02}')

    # 1-year out-of-sample period
    n_splits = int(YEAR / test_length)
    cv = MultipleTimeSeriesCV(n_splits=n_splits,
                              test_period_length=test_length,
                              lookahead=lookahead,
                              train_period_length=train_length)

    predictions = []
    start = time()
    for i, (train_idx, test_idx) in enumerate(cv.split(X=data), 1):
        print(i, end=' ', flush=True)
        lgb_train = lgb_data.subset(used_indices=train_idx.tolist(),
                                    params=params).construct()

        model = lgb.train(params=params,
                          train_set=lgb_train,
                          num_boost_round=num_boost_round)

        test_set = data.iloc[test_idx, :]
        y_test = test_set.loc[:, label].to_frame('y_test')
        y_pred = model.predict(test_set.loc[:, model.feature_name()])
        predictions.append(y_test.assign(prediction=y_pred))

    if position == 0:
        test_predictions = (pd.concat(predictions)
                            .rename(columns={'prediction': position}))
    else:
        test_predictions[position] = pd.concat(predictions).prediction

by_day = test_predictions.groupby(level='date')
for position in range(10):
    if position == 0:
        ic_by_day = by_day.apply(lambda x: spearmanr(
            x.y_test, x[position])[0]).to_frame()
    else:
        ic_by_day[position] = by_day.apply(
            lambda x: spearmanr(x.y_test, x[position])[0])
print(ic_by_day.describe())
test_predictions.to_hdf(store, f'lgb/test/{lookahead:02}')


Position: 00
---------{dates}
1 

---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 01
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 02
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 03
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 04
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 05
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 06
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 07
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 08
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 09
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4                 0           1           2           3           4           5  \
count  252.000000  252.000000  252.000000  252.00000

## Generate CatBoost predictions

In [146]:
lookaheads = [1, 5, 21]

In [147]:
label_dict = dict(zip(lookaheads, labels))

### Model Configuration

In [148]:
lookahead = 1
store = Path('data/predictions.h5')

### Get Data

In [149]:
data = pd.read_hdf('data.h5', 'model_data').sort_index()

In [150]:
labels = sorted(data.filter(like='_fwd').columns)
features = data.columns.difference(labels).tolist()
label = f'r{lookahead:02}_fwd'

In [151]:
data = data.loc[idx[:, '2010':], features + [label]].dropna()

In [152]:
for feature in categoricals:
    data[feature] = pd.factorize(data[feature], sort=True)[0]

In [153]:
cat_cols_idx = [data.columns.get_loc(c) for c in categoricals]

In [154]:
catboost_data = Pool(label=data[label],
                     data=data.drop(label, axis=1),
                     cat_features=cat_cols_idx)

### Generate predictions

In [155]:
catboost_ic = pd.read_hdf('data/model_tuning.h5', 'catboost/ic')
catboost_ic_avg = pd.read_hdf('data/model_tuning.h5', 'catboost/daily_ic')

In [156]:
def get_cb_params(data, t=5, best=0):
    param_cols = scope_params[1:] + catboost_train_params + ['boost_rounds']
    df = data[data.lookahead==t].sort_values('ic', ascending=False).iloc[best]
    return df.loc[param_cols]

In [157]:
for position in range(10):
    params = get_cb_params(catboost_ic_avg,
                    t=lookahead,
                    best=position)
    
    params = params.to_dict()
    
    for p in ['max_depth', 'min_child_samples']:
        params[p] = int(params[p])
    train_length = int(params.pop('train_length'))
    test_length = int(params.pop('test_length'))
    num_boost_round = int(params.pop('boost_rounds'))
    # params['task_type'] = 'GPU'

    print(f'\nPosition: {position:02}')
    
    # 1-year out-of-sample period
    n_splits = int(YEAR / test_length)
    cv = MultipleTimeSeriesCV(n_splits=n_splits,
                              test_period_length=test_length,
                              lookahead=lookahead,
                              train_period_length=train_length)

    predictions = []
    start = time()
    for i, (train_idx, test_idx) in enumerate(cv.split(X=data), 1):
        print(i, end=' ', flush=True)
        train_set = catboost_data.slice(train_idx.tolist())

        model = CatBoostRegressor(**params)
        model.fit(X=train_set)

        test_set = data.iloc[test_idx, :]
        y_test = test_set.loc[:, label].to_frame('y_test')
        y_pred = model.predict(test_set.loc[:, model.feature_names_])
        predictions.append(y_test.assign(prediction=y_pred))

    if position == 0:
        test_predictions = (pd.concat(predictions)
                            .rename(columns={'prediction': position}))
    else:
        test_predictions[position] = pd.concat(predictions).prediction

by_day = test_predictions.groupby(level='date')
for position in range(10):
    if position == 0:
        ic_by_day = by_day.apply(lambda x: spearmanr(x.y_test, x[position])[0]).to_frame()
    else:
        ic_by_day[position] = by_day.apply(lambda x: spearmanr(x.y_test, x[position])[0])
print(ic_by_day.describe())
test_predictions.to_hdf(store, f'catboost/test/{lookahead:02}')


Position: 00
---------{dates}
1 Learning rate set to 0.075661
0:	learn: 0.0333313	total: 27.5ms	remaining: 27.5s
1:	learn: 0.0333089	total: 36.1ms	remaining: 18s
2:	learn: 0.0332628	total: 42.5ms	remaining: 14.1s
3:	learn: 0.0331979	total: 48.5ms	remaining: 12.1s
4:	learn: 0.0331672	total: 53.2ms	remaining: 10.6s
5:	learn: 0.0331434	total: 59.9ms	remaining: 9.93s
6:	learn: 0.0331238	total: 65.7ms	remaining: 9.33s
7:	learn: 0.0331079	total: 70.7ms	remaining: 8.76s
8:	learn: 0.0330593	total: 76.8ms	remaining: 8.46s
9:	learn: 0.0330376	total: 82ms	remaining: 8.11s
10:	learn: 0.0330229	total: 86.8ms	remaining: 7.81s
11:	learn: 0.0330076	total: 94.2ms	remaining: 7.75s
12:	learn: 0.0329738	total: 97.8ms	remaining: 7.42s
13:	learn: 0.0329523	total: 104ms	remaining: 7.35s
14:	learn: 0.0329420	total: 111ms	remaining: 7.29s
15:	learn: 0.0329295	total: 116ms	remaining: 7.15s
16:	learn: 0.0329134	total: 121ms	remaining: 7.01s
17:	learn: 0.0329033	total: 127ms	remaining: 6.92s
18:	learn: 0.0328864